In [12]:
using Pkg
Pkg.add("WAV")
Pkg.add("Glob")

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed Glob ─ v1.2.0
  Updating `~/.julia/environments/v1.2/Project.toml`
  [c27321d9] + Glob v1.2.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [c27321d9] + Glob v1.2.0


In [43]:
Pkg.add("PyCall")
Pkg.add("DataStructures")
Pkg.add("SparseArrays")
Pkg.add("ProgressBars")
Pkg.add("DelimitedFiles")
Pkg.add("Plots")
Pkg.add("Images")


 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [438e738f] + PyCall v1.91.3
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [864edb3b] + DataStructures v0.17.10
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [2f01184e] + SparseArrays 
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed ProgressBars ─ v0.6.0
  Updating `~/.julia/environments/v1.2/Project.toml`
  [49802e3a] + ProgressBars v0.6.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [49802e3a] + ProgressBars v0.6.0
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [8bb1440f] + DelimitedFiles 
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package ver

┌ Info: Recompiling stale cache file /homes/i18momay/.julia/compiled/v1.2/DataStructures/xKiwJ.ji for DataStructures [864edb3b-99cc-5e75-8d2d-829cb0a9cfe8]
└ @ Base loading.jl:1240
┌ Info: Precompiling ProgressBars [49802e3a-d2f1-5c88-81d8-b72133a6f568]
└ @ Base loading.jl:1242
┌ Info: Recompiling stale cache file /homes/i18momay/.julia/compiled/v1.2/PyCall/GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1240
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1242


In [2]:
using DataStructures
using SparseArrays
using LinearAlgebra
using ProgressBars
using PyCall
using Plots

In [63]:
Pkg.add("FFTW")

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [7a1cc6ca] + FFTW v1.1.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [3]:
function init_dictionary(n::Int, K::Int)
    """
    Initialize the dictionary.
    Args: 
        n: dimension of input signal
        k = number of atoms in the dictionary
    """
    # D must be a full-rank matrix
    D = rand(n, K)
    while rank(D) != min(n, K)
        D = rand(n, K)
    end

    @inbounds for k in 1:K
        D[:, k] ./= norm(@view(D[:, k]))
    end
    return D
end

init_dictionary (generic function with 1 method)

In [4]:
# The implementation is referencing the wikipedia page
# https://en.wikipedia.org/wiki/Matching_pursuit#The_algorithm

const default_max_iter = 20
const default_tolerance = 1e-6


function SparseArrays.sparsevec(d::DefaultDict, m::Int)
    SparseArrays.sparsevec(collect(keys(d)), collect(values(d)), m)
end


function matching_pursuit_(data::AbstractVector, dictionary::AbstractMatrix,
                           max_iter::Int, tolerance::Float64)
    n_atoms = size(dictionary, 2)

    residual = copy(data)

    xdict = DefaultDict{Int, Float64}(0.)
    for i in 1:max_iter
        if norm(residual) < tolerance
            return sparsevec(xdict, n_atoms)
        end

        # find an atom with maximum inner product
        products = dictionary' * residual
        _, maxindex = findmax(abs.(products))
        maxval = products[maxindex]
        atom = dictionary[:, maxindex]

        # c is the length of the projection of data onto atom
        a = maxval / sum(abs2, atom)  # equivalent to maxval / norm(atom)^2
        residual -= atom * a

        xdict[maxindex] += a
    end
    return sparsevec(xdict, n_atoms)
end


"""
    matching_pursuit(data::Vector, dictionary::AbstractMatrix;
                     max_iter::Int = $default_max_iter,
                     tolerance::Float64 = $default_tolerance)
Find ``x`` such that ``Dx = y`` or ``Dx ≈ y`` where y is `data` and D is `dictionary`.
```
# Arguments
* `max_iter`: Hard limit of iterations
* `tolerance`: Exit when the norm of the residual < tolerance
```
"""
function matching_pursuit(data::AbstractVector, dictionary::AbstractMatrix;
                          max_iter::Int = default_max_iter,
                          tolerance = default_tolerance)

    if tolerance <= 0
        throw(ArgumentError("`tolerance` must be > 0"))
    end

    if max_iter <= 0
        throw(ArgumentError("`max_iter` must be > 0"))
    end

    if size(data, 1) != size(dictionary, 1)
        throw(ArgumentError(
            "Dimensions must match: `size(data, 1)` and `size(dictionary, 1)`."
        ))
    end

    matching_pursuit_(data, dictionary, max_iter, tolerance)
end


"""
    matching_pursuit(data::AbstractMatrix, dictionary::AbstractMatrix;
                     max_iter::Int = $default_max_iter,
                     tolerance::Float64 = $default_tolerance)
Find ``X`` such that ``DX = Y`` or ``DX ≈ Y`` where Y is `data` and D is `dictionary`.
```
# Arguments
* `max_iter`: Hard limit of iterations
* `tolerance`: Exit when the norm of the residual < tolerance
```
"""
function matching_pursuit(data::AbstractMatrix, dictionary::AbstractMatrix;
                          max_iter::Int = default_max_iter,
                          tolerance::Float64 = default_tolerance)
    K = size(dictionary, 2)
    N = size(data, 2)

    X = spzeros(K, N)

    for i in 1:N
        X[:, i] = matching_pursuit(
            vec(data[:, i]),
            dictionary,
            max_iter = max_iter,
            tolerance = tolerance
        )
    end
    return X
end

matching_pursuit

In [5]:
function K_SVD(Y,niter_KSVD,n_atoms)
    """
    Computes the K-SVD algorithm.
    
    Args:
    Y - 
    niter_KSVD: number of iterations for the algorithm
    n_atoms - number of atoms in the dictionary
    returns: Dictionary (D) and Sparse coefficients (X) 
    
    """
    D = init_dictionary(size(Y,1),n_atoms)
    X = matching_pursuit(Y,D)
    for i in ProgressBar(1:niter_KSVD)
        X = matching_pursuit(Y,D)
        for k=1:n_atoms
            Xk = X[k,:]
            all(iszero,Xk)&&continue
            wk = findall(!iszero,Xk)
            indices = [j for j=1:size(D,2) if j!=k]
            Ek = Y - D[:,indices]*X[indices,:]
            Ωk = sparse(wk,1:length(wk),ones(length(wk)),size(Y,2),length(wk))
            U, S, V= svd(Ek*Ωk, full=true)
            D[:,k]=U[:,1]
            X[k,wk] = V[:,1]*S[1]
        end
    end
    return D,X
end

K_SVD (generic function with 1 method)

In [8]:
using WAV
using Glob
using FFTW

crop_size = 110250
Y = zeros(crop_size,622)
dir = "dataset1/"
for (i,file) in enumerate(glob("*.wav"))
    x,f = wavread(file)
    if length(x)<crop_size
        x = [x;zeros(crop_size-length(x))]
    else
        x = x[1:crop_size]
    end
    Y[:,i] = x
end
    

In [12]:
y = Y[:,1]
D,X = K_SVD(y[findmax(y)[2]-6000:findmax(y)[2]+6020],50,256)

100.0%┣██████████████████████████████████████████████████████████████┫ 50/50 [05:17<00:00, 0.2 it/s]


([0.005905981549478146 0.000605899833697591 … 0.0045301073029107705 0.012889371834577452; 0.006028933002479242 0.010813532556569166 … 0.0002763712308579338 0.007185824492915112; … ; 0.006619791967986195 0.006780331856643526 … 0.005134017935574679 0.001233689667777589; 0.014418664389293312 0.007612604874839575 … 0.007795986937378847 0.003580751503581158],   [3  ]  =  12.8795
  [51 ]  =  0.00241869
  [73 ]  =  0.000670669
  [95 ]  =  0.00178349
  [165]  =  0.00150925
  [194]  =  0.00981407
  [208]  =  0.162918
  [216]  =  0.000504619)

In [15]:
sign_reconstructed = D*X
wavwrite(sign_reconstructed, "reconstructed.wav", Fs=44100)

In [17]:
S = zeros(size(y))
S[3] = 12.8795
S[51] = 0.00241869
S[73] = 0.000670669
S[95] = 0.00178349
S[165] = 0.00150925
S[194] = 0.00981407
S[208] = 0.162918
S[216] = 0.000504619
wavwrite(5*S, "S.wav", Fs=44100)

In [ ]:
# Find D and X such that Y ≈ DX

println("||Y - D * X|| = $(norm(Y - D * X))")

println("The ratio of zero elemnts in the matrix X: ",
        sum(X .== 0) / length(X))